In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from psql_models import read_table_async,DiscussionPoint,Meeting,Speaker,Output


In [4]:

discussion_points_df = await read_table_async(DiscussionPoint)
meetings_df = await read_table_async(Meeting)
speakers_df = await read_table_async(Speaker)

In [5]:
import pandas as pd

In [6]:
from sampling import fetch_joined_data,WeightedSampler

In [7]:
joined_df = await fetch_joined_data()
joined_df['meeting_time'] = pd.to_datetime(joined_df['meeting_timestamp']).dt.strftime('%Y-%m-%d')

In [8]:
# Set display options to show all content
pd.set_option('display.max_colwidth', 100)  # Show full contents of each column
pd.set_option('display.max_rows', 20)      # Show all rows
pd.set_option('display.max_columns', 20)   # Show all columns
pd.set_option('display.width', 100)         # Don't wrap wide columns

In [9]:
len(joined_df)

3194

In [10]:
joined_df.groupby('speaker_name').count().sort_values(by='summary_index', ascending=False).index.tolist()


['Dmitriy Grankin',
 'Olga Nemirovskaya',
 'Dmitry Grankin',
 'Sergey Ryabenko',
 'Andrey Pisankin',
 'DG',
 'nan',
 'Alex Shevliakov',
 'Dmitrii Bashkirov',
 'David Sterry',
 'Rick Tousseyn',
 'Nikolay',
 'nan (0)',
 'Ilia Semukhin',
 'Matt Lewis',
 'Karine Stepanyan',
 'Olga Miller',
 'Павел Коркодинов',
 'Nick Frolov',
 'Murat Sultanov',
 'Robert Hangu',
 'Xut tuX',
 'Dmitrii Chistov',
 'nan (1)',
 'Ahmed Abdelaziz',
 'Julia Zakharova',
 'Илья Дзенски',
 'Дмитрий Гранкин',
 'Igor Vinidiktov',
 'Сергей Жилко',
 'Shota Arabuli',
 'Olga Mykhoparkina',
 'KS',
 'Igor Bessonov',
 'Олег',
 'Andy Black',
 'Sergio Goriachev',
 'ЮБ',
 'Tatiana Illarionova-Zervas',
 'Mayank Tayal',
 'Ethan Noah',
 'Daniil Andreev',
 'Eugene Tartakovsky',
 'MJ',
 'Nadya Gorodetskaya',
 'Özay Demirezen',
 'nan (3)',
 'nan (2)',
 'Lidiia Abramova',
 'Александр Мелихов',
 'Jenish Mursidinov',
 'Tatiana Sukhova',
 'Lara Vargas',
 'Vasilii Glebov',
 'Umar Lateef',
 'Сергей',
 'XT',
 'OM',
 'Dmtiry Grankin',
 'Sergey

In [11]:
coworkers = ['Dmitriy Grankin','Sergey Ryabenko','Olga Nemirovskaya','Dmitry Grankin']

In [12]:
selected_speakers = ['Igor Bessonov']


In [13]:
filtered_df = joined_df[joined_df['speaker_name'].isin(selected_speakers)]

In [14]:
filtered_df = joined_df[~joined_df['speaker_name'].isin(coworkers)]

In [38]:
sampler = WeightedSampler(
    df=filtered_df.drop(columns=['other_speakers']),
    date_column='meeting_timestamp',
    decay_factor=0.2
)

Building vector index...


Batches:   0%|          | 0/36 [00:00<?, ?it/s]

In [24]:

# Sample recent data
sampled_df = sampler.sample(
    n_samples=1000,
    mode='recency',
    recency_weight=1.0,
    similarity_weight=0.0
).sort_values('meeting_timestamp', ascending=True).reset_index(drop=True)

In [37]:
sampled_df

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,meeting_time,filter_score,recency_score,similarity_score,combined_score
0,17,The assistant is discussed as a tool for user interaction within the dashboard.,The team is considering how the assistant can help users filter and interact with call data effe...,,ассистент,concept,288381d0-3ca7-4601-99b9-b42da8cd30d3,2024-08-27 17:42:45.726,Alex Shevliakov,2024-08-27,1.0,2.394326e-07,0.0,0.000875
1,15,"The dashboard is a central topic of discussion in the meeting, focusing on its functionalities a...",The team discussed the need for filtering options and the assistant's integration with the dashb...,,дешборд,concept,288381d0-3ca7-4601-99b9-b42da8cd30d3,2024-08-27 17:42:45.726,Alex Shevliakov,2024-08-27,1.0,2.394326e-07,0.0,0.000875
2,20,Payments are discussed in relation to the dashboard's functionalities.,"Alex mentioned the need to integrate payment methods, specifically referencing Stripe.",,платежи,concept,288381d0-3ca7-4601-99b9-b42da8cd30d3,2024-08-27 17:42:45.726,Alex Shevliakov,2024-08-27,1.0,2.394326e-07,0.0,0.000875
3,8,Integrate payment methods and billing options into the platform to facilitate user transactions.,Consider using Stripe for payment processing and ensure users can manage their payment methods a...,Alex Shevliakov: ждет так еще добавил биллинги нужно был пилинги с с планом добавление payment ...,Payment Methods and Billing Integration,task,288381d0-3ca7-4601-99b9-b42da8cd30d3,2024-08-27 17:42:45.726,Alex Shevliakov,2024-08-27,1.0,2.394326e-07,0.0,0.000875
4,23,Framer is mentioned as a potential tool for building the landing page.,Alex discussed the advantages and disadvantages of using Framer for creating the landing page co...,,фреймер,concept,288381d0-3ca7-4601-99b9-b42da8cd30d3,2024-08-27 17:42:45.726,Alex Shevliakov,2024-08-27,1.0,2.394326e-07,0.0,0.000875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,16,WhatsApp is a messaging platform that Özay mentions in the context of challenges in recording ca...,"Özay discusses the difficulty of recording calls made through WhatsApp, highlighting a gap in cu...",Dmitry Grankin: with fred fred you know okay yeah so we finished with fred uh we went to ic uh ...,WhatsApp,product,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,2024-10-24,1.0,2.138654e-02,0.0,0.000937
996,2,Özay discussed the long sales cycles in the manufacturing sector and the difficulties faced in s...,"Özay mentioned that they are in discussions with multiple manufacturing companies, but the sales...",Özay Demirezen: And also the area that we are trying to sell stuff is manufacturing long sales ...,Challenges in the manufacturing sector,concern,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,2024-10-24,1.0,2.138654e-02,0.0,0.000937
997,15,Vexa is a product that Özay has tried for recording meetings.,"Özay mentions trying Vexa for one meeting, indicating his interest in solutions for recording an...","Özay Demirezen: But I've seen similar examples, even like I've been using Blue Dot or something.",Vexa,product,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,2024-10-24,1.0,2.138654e-02,0.0,0.000937
998,6,Özay Demirezen is a speaker in the meeting who shares insights on product management and challen...,Özay discusses the difficulties of recording phone calls and the challenges faced in the manufac...,"Özay Demirezen: Hi, I'm in Barcelona. I moved to Barcelona like seven, eight months ago, I thin...",Özay Demirezen,person,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,2024-10-24,1.0,2.138654e-02,0.0,0.000937


In [26]:
context = sampled_df.reset_index()[['index','speaker_name','topic_name','summary']].to_csv()


In [27]:
from core import count_tokens
count_tokens(context)

31323

In [28]:
from core import generic_call_stream,system_msg,user_msg,assistant_msg


In [29]:
from prompts import Prompts
prompts = Prompts()

In [30]:
prompts.perplexity

'You are Vexa, a helpful search assistant trained by Vexa AI. Your task is to deliver a concise and accurate response to a user\'s query, drawing from the given search results. Your answer must be precise, of high-quality, and written by an expert using an unbiased and journalistic tone. It is EXTREMELY IMPORTANT to directly answer the query. NEVER say "based on the search results" or start your answer with a heading or title. Get straight to the point. Your answer must be written in the same language as the query, even if language preference is different.\n\nYou MUST cite the most relevant search results that answer the query. Do not mention any irrelevant results. You MUST ADHERE to the following instructions for citing search results: - to cite a search result, enclose its index located above the summary with brackets at the end of the corresponding sentence, for example "Ice is less dense than water[1][2]." or "Paris is the capital of France[1][4][5]." - NO SPACE between the last w

In [31]:
misspelled = ['Vexa.ai','TL;DV']

In [34]:
context

',index,speaker_name,topic_name,summary\n0,0,Alex Shevliakov,ассистент,The assistant is discussed as a tool for user interaction within the dashboard.\n1,1,Alex Shevliakov,дешборд,"The dashboard is a central topic of discussion in the meeting, focusing on its functionalities and user interface."\n2,2,Alex Shevliakov,платежи,Payments are discussed in relation to the dashboard\'s functionalities.\n3,3,Alex Shevliakov,Payment Methods and Billing Integration,Integrate payment methods and billing options into the platform to facilitate user transactions.\n4,4,Alex Shevliakov,фреймер,Framer is mentioned as a potential tool for building the landing page.\n5,5,Alex Shevliakov,PostHoc,PostHoc is referenced as an analytics tool that can help track user activity and campaign effectiveness.\n6,6,Alex Shevliakov,рекординги,Recordings are mentioned in the context of the dashboard\'s functionalities.\n7,7,Alex Shevliakov,Берлин,Berlin is mentioned as a travel destination in the context of a trip take

In [35]:
r = await generic_call_stream([
    system_msg(f'''{prompts.perplexity}. 
               You user is Dmitry Grankin, who is your user, who also participates the meetings in the context. 
               '''),
    system_msg(f'''Today is {pd.to_datetime('today').strftime('%Y-%m-%d')}.'''),
    #system_msg('''Frequently misspelled names and terms: {misspelled}. Replace misspelled company names and terms if found in context.'''),
    
    user_msg(f'search results: {context}. Give comprehensive answer based on the context only. Interprete user question based on the context and the context based on the question.'),
    
    user_msg('''summarize the context''')
],model='default')

The context revolves around discussions among various speakers about product development, marketing strategies, and the integration of AI technologies in different sectors, particularly focusing on tools like Vexa, which assists in meeting transcription and analysis. Key topics include user experience, the importance of effective design, and the challenges of onboarding users. There are also discussions about competitive analysis, marketing strategies for launching products on platforms like Product Hunt, and the potential for collaborations with educational institutions and other companies.

Speakers share insights on their experiences with different technologies, including FastAPI, PostgreSQL, and various AI tools, while addressing issues related to user engagement, data management, and the functionality of their products. The conversations highlight the need for structured communication, effective marketing strategies, and the importance of understanding user needs to enhance produc

In [36]:
from IPython.display import Markdown
Markdown(r)

The context revolves around discussions among various speakers about product development, marketing strategies, and the integration of AI technologies in different sectors, particularly focusing on tools like Vexa, which assists in meeting transcription and analysis. Key topics include user experience, the importance of effective design, and the challenges of onboarding users. There are also discussions about competitive analysis, marketing strategies for launching products on platforms like Product Hunt, and the potential for collaborations with educational institutions and other companies.

Speakers share insights on their experiences with different technologies, including FastAPI, PostgreSQL, and various AI tools, while addressing issues related to user engagement, data management, and the functionality of their products. The conversations highlight the need for structured communication, effective marketing strategies, and the importance of understanding user needs to enhance product offerings and drive adoption. Overall, the meetings focus on leveraging technology to improve business processes and user interactions.

In [ ]:
Markdown(r)

Olga Nemirovskaya discusses several aspects of SEO in the context of their marketing strategies. She emphasizes the importance of internal linking and domain authority as critical factors for SEO success. Olga explains that internal linking helps improve search rankings and that building domain authority through quality backlinks is essential for enhancing visibility.

Additionally, she mentions the need for clear messaging on their website to improve client understanding and engagement, which ties into SEO practices. Olga also highlights the significance of using tools like Surfer SEO for optimizing content based on competitive analysis, indicating that they have been using it successfully for several years to improve their articles' performance in search engines[1][2][3][4].